https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings<br>
: problem page

http://blog.kaggle.com/2013/01/17/getting-started-with-pandas-predicting-sat-scores-for-new-york-city-schools/<br>
: data cleaning



In [3]:
%matplotlib inline

# Data description

Airbnb challenges you to predict in which country a new user will make his or her first booking.

Train.csv
: the test set of users

    id: user id
    
    date_account_created: the date of account creation
    
    timestamp_first_active: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a **user can search before signing up**
    
    date_first_booking: date of first booking
    
    gender
    age
    signup_method
    
    signup_flow: the page a user came to signup up from
    
    language: international language preference
    
    affiliate_channel: what kind of paid marketing
    
    affiliate_provider: where the marketing is e.g. google, craigslist, other
    
    first_affiliate_tracked: whats the first marketing the user interacted with before the signing up
    signup_app
    
    first_device_type
    first_browser
    
    country_destination: this is the target variable you are to predict

Sessions.csv
: web sessions log for users

    user_id: to be joined with the column 'id' in users table
    action
    action_type
    action_detail
    device_type
    secs_elapsed

Age_gender_bkts.csv
: summary statistics of users' age group, gender, country of destination



# Data's shape?

In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [40]:
!head data/age_gender_bkts.csv -n 25

age_bucket,country_destination,gender,population_in_thousands,year
100+,AU,male,1.0,2015.0
95-99,AU,male,9.0,2015.0
90-94,AU,male,47.0,2015.0
85-89,AU,male,118.0,2015.0
80-84,AU,male,199.0,2015.0
75-79,AU,male,298.0,2015.0
70-74,AU,male,415.0,2015.0
65-69,AU,male,574.0,2015.0
60-64,AU,male,636.0,2015.0
55-59,AU,male,714.0,2015.0
50-54,AU,male,778.0,2015.0
45-49,AU,male,778.0,2015.0
40-44,AU,male,820.0,2015.0
35-39,AU,male,797.0,2015.0
30-34,AU,male,881.0,2015.0
25-29,AU,male,895.0,2015.0
20-24,AU,male,820.0,2015.0
15-19,AU,male,768.0,2015.0
10-14,AU,male,743.0,2015.0
5-9,AU,male,784.0,2015.0
0-4,AU,male,824.0,2015.0
100+,AU,female,4.0,2015.0
95-99,AU,female,25.0,2015.0
90-94,AU,female,94.0,2015.0


In [2]:
!head data/sample_submission_NDF.csv

id,country
5uwns89zht,NDF
jtl0dijy2j,NDF
xx0ulgorjt,NDF
6c6puo6ix0,NDF
czqhjk3yfe,NDF
szx28ujmhf,NDF
guenkfjcbq,NDF
tkpq0mlugk,NDF
3xtgd5p9dn,NDF


In [3]:
!head data/test_users.csv

id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
5uwns89zht,2014-07-01,20140701000006,,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
jtl0dijy2j,2014-07-01,20140701000051,,-unknown-,,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
xx0ulgorjt,2014-07-01,20140701000148,,-unknown-,,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
6c6puo6ix0,2014-07-01,20140701000215,,-unknown-,,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
czqhjk3yfe,2014-07-01,20140701000305,,-unknown-,,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari
szx28ujmhf,2014-07-01,20140701000336,,FEMALE,28.0,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome
guenkfjcbq,2014-07-01,20140701000514,,MALE,48.0,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-
tkpq0mlugk,2014-07-01,201

In [4]:
!head data/countries.csv

country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language ,language_levenshtein_distance
AU,-26.853388,133.27516,15297.744,7741220.0,eng,0.0
CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.0
DE,51.165707,10.452764,7879.568,357022.0,deu,72.61
ES,39.896027,-2.4876945,7730.724,505370.0,spa,92.25
FR,46.232193,2.209667,7682.945,643801.0,fra,92.06
GB,54.63322,-3.4322774,6883.659,243610.0,eng,0.0
IT,41.87399,12.564167,8636.631,301340.0,ita,89.4
NL,52.133057,5.29525,7524.3203,41543.0,nld,63.22
PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45


In [5]:
!head data/train_users_2.csv

id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
gxn3p5htnn,2010-06-28,20090319043255,,-unknown-,,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
820tgsjxq7,2011-05-25,20090523174809,,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,

In [6]:
!head data/sessions.csv

user_id,action,action_type,action_detail,device_type,secs_elapsed
d1mm9tcy42,lookup,,,Windows Desktop,319.0
d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
d1mm9tcy42,lookup,,,Windows Desktop,301.0
d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
d1mm9tcy42,lookup,,,Windows Desktop,435.0
d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,7703.0
d1mm9tcy42,lookup,,,Windows Desktop,115.0
d1mm9tcy42,personalize,data,wishlist_content_update,Windows Desktop,831.0
d1mm9tcy42,index,view,view_search_results,Windows Desktop,20842.0


Data is too complex to see in raw.

In [3]:
df_age_gender = pd.read_csv('../data/age_gender_bkts.csv')
df_train = pd.read_csv('../data/train_users_2.csv')
df_country = pd.read_csv('../data/countries.csv')
df_session = pd.read_csv('../data/sessions.csv')
df_test = pd.read_csv('../data/test_users.csv')
df_sample = pd.read_csv('../data/sample_submission_NDF.csv')

In [105]:
df_train.shape

(213451, 16)

In [10]:
df_train[0:5]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [34]:
df_train[0:5].T

,0,1,2,3,4
id,gxn3p5htnn,820tgsjxq7,4ft3gnwmtx,bjjt8pjhuk,87mebub9p4
date_account_created,2010-06-28,2011-05-25,2010-09-28,2011-12-05,2010-09-14
timestamp_first_active,20090319043255,20090523174809,20090609231247,20091031060129,20091208061105
date_first_booking,NaN,NaN,2010-08-02,2012-09-08,2010-02-18
gender,-unknown-,MALE,FEMALE,FEMALE,-unknown-
age,NaN,38,56,42,41
signup_method,facebook,facebook,basic,facebook,basic
signup_flow,0,0,3,0,0
language,en,en,en,en,en
affiliate_channel,direct,seo,direct,direct,direct


In [106]:
df_test.shape

(62096, 15)

In [11]:
df_test[0:5]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [52]:
df_test[0:5].T

,0,1,2,3,4
id,5uwns89zht,jtl0dijy2j,xx0ulgorjt,6c6puo6ix0,czqhjk3yfe
date_account_created,2014-07-01,2014-07-01,2014-07-01,2014-07-01,2014-07-01
timestamp_first_active,20140701000006,20140701000051,20140701000148,20140701000215,20140701000305
date_first_booking,NaN,NaN,NaN,NaN,NaN
gender,FEMALE,-unknown-,-unknown-,-unknown-,-unknown-
age,35,NaN,NaN,NaN,NaN
signup_method,facebook,basic,basic,basic,basic
signup_flow,0,0,0,0,0
language,en,en,en,en,en
affiliate_channel,direct,direct,direct,direct,direct


In [12]:
df_country[0:5]

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801,fra,92.06


In [56]:
df_age_gender[0:25]

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1,2015
1,95-99,AU,male,9,2015
2,90-94,AU,male,47,2015
3,85-89,AU,male,118,2015
4,80-84,AU,male,199,2015
5,75-79,AU,male,298,2015
6,70-74,AU,male,415,2015
7,65-69,AU,male,574,2015
8,60-64,AU,male,636,2015
9,55-59,AU,male,714,2015


In [104]:
df_session.shape

(10567737, 6)

In [14]:
df_session[0:5]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435


In [107]:
df_sample.shape

(62096, 2)

In [15]:
df_sample[0:5]

,id,country
0,5uwns89zht,NDF
1,jtl0dijy2j,NDF
2,xx0ulgorjt,NDF
3,6c6puo6ix0,NDF
4,czqhjk3yfe,NDF


## Missing value

In [44]:
def getNullCount(df):
    cols = df.columns.values
    
    nrows = df.shape[0]
    print('nrows: ', nrows)
    
    for col in cols:
        print('>>>> ' + col)
        nullCnt = sum(df[col].isnull())
        print('sum: ', nullCnt, ' ==> ', nullCnt*100/nrows, '%')


In [37]:
def getDataFiltering(df):
    cols = df.columns.values
    cols = cols[np.where(cols != 'date_first_booking')]
    
    dfTemp = df[cols]
    dfTemp = dfTemp.dropna(how='any')
    df_temp = (dfTemp['gender']=='-unknown-') | (dfTemp['first_browser']=='-unknown-')

    return dfTemp[~df_temp]

In [19]:
np.where(df_train.columns.values != 'date_first_booking')

(array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),)

In [20]:
df_train.columns.values[np.where(df_train.columns.values != 'date_first_booking')]

array(['id', 'date_account_created', 'timestamp_first_active', 'gender',
       'age', 'signup_method', 'signup_flow', 'language',
       'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination'], dtype=object)

In [38]:
dfTemp = getDataFiltering(df_train)
dfTemp.shape[0]

97979

In [41]:
dfTemp.to_csv('../data/train_step1.csv')

In [42]:
dfTemp = getDataFiltering(df_test)
dfTemp.shape[0]
dfTemp.to_csv('../data/test_step1.csv')

In [43]:
dfTemp

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,FEMALE,35,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
5,szx28ujmhf,2014-07-01,20140701000336,FEMALE,28,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome
12,iq4kkd5oan,2014-07-01,20140701002539,MALE,24,facebook,0,en,direct,direct,linked,Web,Mac Desktop,Safari
15,eq6fy0m4vc,2014-07-01,20140701003103,FEMALE,56,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
18,i0sc6d3j8s,2014-07-01,20140701003723,FEMALE,33,basic,0,en,direct,direct,omg,Web,Android Tablet,IE
19,br5mcrsqzn,2014-07-01,20140701004052,MALE,31,facebook,0,en,seo,google,linked,Web,Windows Desktop,Chrome
20,rddbczuxx1,2014-07-01,20140701004053,FEMALE,53,basic,0,en,sem-brand,google,linked,Web,Mac Desktop,Safari
22,sxpkaxep8n,2014-07-01,20140701004343,MALE,35,facebook,0,en,seo,google,linked,Web,Windows Desktop,Chrome
25,ovc6nwn6mj,2014-07-01,20140701004654,MALE,34,basic,0,en,direct,direct,linked,Moweb,iPhone,Mobile Safari
26,n10skstp90,2014-07-01,20140701004748,FEMALE,25,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Chrome


In [100]:
print(df_train.shape)
df_train.head(10)

(213451, 16)


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US
8,a1vcnhxeij,2010-01-04,20100104004211,2010-07-29,FEMALE,50,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
9,6uh8zyj2gn,2010-01-04,20100104023758,2010-01-04,-unknown-,46,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,US


In [99]:
cols = df_train.columns.values
dfTemp = df_train.dropna(how='any')
print(dfTemp.shape)
dfTemp.head(10)

(68171, 16)


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US
8,a1vcnhxeij,2010-01-04,20100104004211,2010-07-29,FEMALE,50,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
9,6uh8zyj2gn,2010-01-04,20100104023758,2010-01-04,-unknown-,46,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,US
10,yuuqmid2rp,2010-01-04,20100104194251,2010-01-06,FEMALE,36,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Firefox,US
15,v4d5rl22px,2010-01-07,20100107204555,2010-01-08,FEMALE,33,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,CA
19,gdka1q5ktd,2010-01-10,20100110010817,2010-01-10,FEMALE,29,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,FR


In [47]:
getNullCount(df_session)

nrows:  10567737
>>>> user_id
sum:  34496  ==>  0.32642750288 %
>>>> action
sum:  79626  ==>  0.753482036883 %
>>>> action_type
sum:  1126204  ==>  10.6570025352 %
>>>> action_detail
sum:  1126204  ==>  10.6570025352 %
>>>> device_type
sum:  0  ==>  0.0 %
>>>> secs_elapsed
sum:  136031  ==>  1.28722923366 %


In [45]:
getNullCount(df_test)

nrows:  62096
>>>> id
sum:  0  ==>  0.0 %
>>>> date_account_created
sum:  0  ==>  0.0 %
>>>> timestamp_first_active
sum:  0  ==>  0.0 %
>>>> date_first_booking
sum:  62096  ==>  100.0 %
>>>> gender
sum:  0  ==>  0.0 %
>>>> age
sum:  28876  ==>  46.5021901572 %
>>>> signup_method
sum:  0  ==>  0.0 %
>>>> signup_flow
sum:  0  ==>  0.0 %
>>>> language
sum:  0  ==>  0.0 %
>>>> affiliate_channel
sum:  0  ==>  0.0 %
>>>> affiliate_provider
sum:  0  ==>  0.0 %
>>>> first_affiliate_tracked
sum:  20  ==>  0.0322081937645 %
>>>> signup_app
sum:  0  ==>  0.0 %
>>>> first_device_type
sum:  0  ==>  0.0 %
>>>> first_browser
sum:  0  ==>  0.0 %


In [46]:
getNullCount(df_train)

nrows:  213451
>>>> id
sum:  0  ==>  0.0 %
>>>> date_account_created
sum:  0  ==>  0.0 %
>>>> timestamp_first_active
sum:  0  ==>  0.0 %
>>>> date_first_booking
sum:  124543  ==>  58.3473490403 %
>>>> gender
sum:  0  ==>  0.0 %
>>>> age
sum:  87990  ==>  41.2225756731 %
>>>> signup_method
sum:  0  ==>  0.0 %
>>>> signup_flow
sum:  0  ==>  0.0 %
>>>> language
sum:  0  ==>  0.0 %
>>>> affiliate_channel
sum:  0  ==>  0.0 %
>>>> affiliate_provider
sum:  0  ==>  0.0 %
>>>> first_affiliate_tracked
sum:  6065  ==>  2.84140153946 %
>>>> signup_app
sum:  0  ==>  0.0 %
>>>> first_device_type
sum:  0  ==>  0.0 %
>>>> first_browser
sum:  0  ==>  0.0 %
>>>> country_destination
sum:  0  ==>  0.0 %


## Unique value

In [66]:
def getUnique(df):
    cols = df.columns.values
    tot = df.shape[0]
    print('nrows: ', tot)
    
    for col in cols:
        print('>>>> ' + col)
#         s2 = pd.DataFrame(df[col].unique(), columns=['desc'])
#         print(s2)
#         print('count: ', df[col].nunique())
#         print(pd.DataFrame(df_session.groupby(col)[col].count()).sort(columns=col, ascending=False) )
        val = pd.value_counts(df[col].values.ravel())
        print(val/tot*100)


In [55]:
getUnique(df_session)

nrows:  10567737
>>>> user_id
>>>> action
>>>> action_type
>>>> action_detail
>>>> device_type
>>>> secs_elapsed


In [68]:
getUnique(df_test)

nrows:  62096
>>>> id
lffn4mhlqb    0.00161
20t91k851p    0.00161
1y3183eytu    0.00161
6zo8o845vk    0.00161
s116r96wxl    0.00161
4angg7934o    0.00161
ns9sk22bi7    0.00161
ky1j3atyw9    0.00161
oi03v13qce    0.00161
j3oh2o56fm    0.00161
ktowo98lej    0.00161
lu0qjnc8yx    0.00161
kmq1lsfen5    0.00161
l8lo0efdrq    0.00161
9fev913o6t    0.00161
uthfjgqpob    0.00161
4w2xucsxyz    0.00161
anivmktava    0.00161
847chp04mt    0.00161
itxi5mkum5    0.00161
b02zrrgm8t    0.00161
uqcm1b8fb1    0.00161
plreld2rfb    0.00161
mqdgkdlgsb    0.00161
y9y7k0x98n    0.00161
9wpp0apwj7    0.00161
rpd6koxt32    0.00161
9k9dszmleq    0.00161
asyhv6lru3    0.00161
2zh280pf5b    0.00161
               ...   
f295f8a751    0.00161
6yegdeg9ru    0.00161
ho40fzu0na    0.00161
ta8h41131s    0.00161
d91xqm73l1    0.00161
bo6dl89r0l    0.00161
smxofxtwko    0.00161
os4c9vy2rg    0.00161
g1exlea7l2    0.00161
rls1oe2dr9    0.00161
phhd873no5    0.00161
jxgxlr91a2    0.00161
1kiinowjp1    0.00161
2gqpxmrxkp

In [67]:
getUnique(df_train)

nrows:  213451
>>>> id
51d1errpx3    0.000468
rgxqsar11t    0.000468
04712ytu1m    0.000468
woxkl2eiqq    0.000468
5r6btw2myb    0.000468
9772lzszq9    0.000468
sgwrofjwam    0.000468
eeiffqoz0f    0.000468
n4uf8ngcs5    0.000468
l7zdo5hsip    0.000468
cz6d14chib    0.000468
6dqbne2iw0    0.000468
bp4grfutt6    0.000468
pdlt19n9pv    0.000468
8xyhzxatr7    0.000468
nshohgv6qo    0.000468
35cvu3xosz    0.000468
faqljp0aqz    0.000468
2iagpkjz4q    0.000468
72mwfh0cbl    0.000468
u4b25so8e9    0.000468
uzgr69uts8    0.000468
yiy6zdpp40    0.000468
neimmdgqy2    0.000468
azodrzvjpr    0.000468
2bhaabky1n    0.000468
ivtf919y45    0.000468
pemk0yw2ae    0.000468
f4b1oy1vdk    0.000468
urarfxt99w    0.000468
                ...   
0sy1jm3fls    0.000468
jw4wx49ia1    0.000468
yqx2tt0s2s    0.000468
onutwqws8c    0.000468
pmz4s13hlx    0.000468
j8yno6tsyx    0.000468
uem3akfja5    0.000468
48j5fkt9ru    0.000468
2ozpb6mzcc    0.000468
fm4poossry    0.000468
375buakj12    0.000468
rkqvfxursn 

In [44]:
df_train['id'].unique()

array(['gxn3p5htnn', '820tgsjxq7', '4ft3gnwmtx', ..., '6o3arsjbb4',
       'jh95kwisub', 'nw9fwlyb5f'], dtype=object)

In [19]:
pd.Series(df_train.values.ravel()).unique()

array(['gxn3p5htnn', '2010-06-28', 20090319043255, ..., 20140630235822,
       'nw9fwlyb5f', 20140630235824], dtype=object)

In [20]:
df_train.min(axis=0)

id                             00023iyk9l
date_account_created           2010-01-01
timestamp_first_active     20090319043255
date_first_booking                    NaN
gender                          -unknown-
age                                     1
signup_method                       basic
signup_flow                             0
language                               ca
affiliate_channel                     api
affiliate_provider                  baidu
first_affiliate_tracked               NaN
signup_app                        Android
first_device_type           Android Phone
first_browser                   -unknown-
country_destination                    AU
dtype: object

In [21]:
df_train.max(axis=0)

id                             zzzlylp57e
date_account_created           2014-06-30
timestamp_first_active     20140630235824
date_first_booking                    NaN
gender                              OTHER
age                                  2014
signup_method                      google
signup_flow                            25
language                               zh
affiliate_channel                     seo
affiliate_provider                 yandex
first_affiliate_tracked               NaN
signup_app                            iOS
first_device_type                  iPhone
first_browser                  wOSBrowser
country_destination                 other
dtype: object

In [22]:
df_train.median(axis=0)

timestamp_first_active    2.013091e+13
age                       3.400000e+01
signup_flow               0.000000e+00
dtype: float64

In [28]:
df_train.columns.values

array(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method',
       'signup_flow', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser', 'country_destination'], dtype=object)

In [23]:
# number only
df_train.describe()

,timestamp_first_active,age,signup_flow
count,2.134510e+05,125461.000000,213451.000000
mean,2.013085e+13,49.668335,3.267387
std,9.253717e+09,155.666612,7.637707
min,2.009032e+13,1.000000,0.000000
25%,2.012123e+13,28.000000,0.000000
50%,2.013091e+13,34.000000,0.000000
75%,2.014031e+13,43.000000,0.000000
max,2.014063e+13,2014.000000,25.000000


In [30]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 27.7+ MB
